In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('dati supernovae.csv')

In [6]:
df

,# alpha 0.121851859725,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,# beta 2.46569277393,NaN,NaN,NaN,NaN
1,# delta -0.0363405630486,NaN,NaN,NaN,NaN
2,# M(h=0.7 statistical only) -19.3182761161,NaN,NaN,NaN,NaN
3,# M(h=0.7 with systematics) -19.3081547178,NaN,NaN,NaN,NaN
4,# nome_SN,redshift,mod_dist,mod_dist_err,unknown
...,...,...,...,...,...
580,D-000,"1,017","442,939,707,709",0.171046154322,1.48E+02
581,D-006,"1,315","449,713,577,744",0.187508839367,0.551671793295
582,P-009,0.821,"436,409,387,868",0.194013591972,0.551671793295
583,R-012,"1,215","452,465,209,495",0.560317445151,0.000147652666
